## Setup

### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Installation

In [2]:
from IPython.display import clear_output

In [3]:
!pip install -U ultralytics
!apt-get install unrar

clear_output()

In [4]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.3/112.6 GB disk)


In [5]:
import os
import io
import shutil
import yaml
from ultralytics import YOLO
from ultralytics.cfg import get_cfg
from ultralytics.utils import SETTINGS

### Setting

In [6]:
# remove unused files on gdrive
# base_dir = "drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting"

# shutil.rmtree(f"{base_dir}/data_{model_name}_{epochs}")

# os.remove(f"{base_dir}/data-split-track.rar")
# os.remove(f"{base_dir}/data-split-any.zip")
# os.remove(f"{base_dir}/G1_Grbng_Masuk_Brt_NVR250_LG_20250205070020_20250205070200_1608394.mp4")

In [7]:
base_dir = "drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting"
dataset_name = "data"
model_name = "yolo11n"
epochs = 100

In [8]:
# Set the dataset path
SETTINGS['datasets_dir'] = '/content/data'
SETTINGS['runs_dir'] = f'/content/{base_dir}/{dataset_name}_{model_name}_{epochs}'
SETTINGS['tensorboard'] = True

In [9]:
!yolo settings

JSONDict("/root/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/content/data",
  "weights_dir": "weights",
  "runs_dir": "/content/drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting/data_yolo11n_100",
  "uuid": "569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true,
  "openvino_msg": true
}
💡 Learn more about Ultralytics Settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings


⚠️ Restart session after changes the ultralytics settings ⚠️

## Preprocess

In [9]:
# shutil.rmtree("data")
# shutil.rmtree("runs")

### RAR Dataset

In [ ]:
# shutil.copyfile(f"{base_dir}/data-split.rar", "data-split.rar")

# !unrar x data-split.rar
#
# clear_output()

# os.rename("data-split", "data")

### ZIP Dataset

In [13]:
if os.path.exists("data"):
    shutil.rmtree("data")

if not os.path.exists(f"{dataset_name}.zip"):
    shutil.copyfile(f"{base_dir}/{dataset_name}.zip", f"{dataset_name}.zip")

!unzip -q {dataset_name}.zip

if dataset_name != "data":
    os.rename(dataset_name, "data")

## Train

In [14]:
with open("data/data.yaml", "r") as file:
    data_yaml = yaml.safe_load(file)
    print(data_yaml)

if data_yaml["path"] == ".":
    data_yaml["path"] = "data"

with io.open("data/data.yaml", 'w', encoding='utf8') as file:
    yaml.dump(data_yaml, file, default_flow_style=False, allow_unicode=True)

{'names': {0: 'bicycle', 1: 'motorbike', 2: 'car', 3: 'truck', 4: 'bus', 5: 'person'}, 'path': '.', 'test': 'test.txt', 'train': 'train.txt', 'val': 'val.txt'}


In [16]:
# Load a model
model = YOLO(f"{model_name}.yaml")                    # build a new model from YAML
model = YOLO(f"{model_name}.pt")                      # load a pretrained model (recommended for training)
model = YOLO(f"{model_name}.yaml").load(f"{model_name}.pt") # build from YAML and transfer weights

# Train the model
results = model.train(data="/content/data/data.yaml", epochs=epochs, imgsz=640)

Transferred 499/499 items from pretrained weights
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pe

train: Scanning data/labels/train... 2483 images, 702 backgrounds, 0 corrupt: 100%|██████████| 3185/3185 [00:01<00:00, 2616.80it/s]


train: New cache created: data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1277.2±800.1 MB/s, size: 119.1 KB)


val: Scanning data/labels/val... 673 images, 237 backgrounds, 0 corrupt: 100%|██████████| 910/910 [00:00<00:00, 1057.34it/s]

val: New cache created: data/labels/val.cache


...

    100/100       3.8G     0.3453     0.2219     0.7802          0        640: 100%|██████████| 200/200 [01:01<00:00,  3.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:06<00:00,  4.60it/s]

                   all        910       1666      0.981      0.982      0.994      0.934



100 epochs completed in 1.965 hours.
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting/data-split_yolo11n_100/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from /content/drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting/data-split_yolo11n_100/detect/train/weights/best.pt, 5.4MB

Validating /content/drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting/data-split_yolo11n_100/detect/train/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:07<00:00,  3.67it/s]


                   all        910       1666      0.978      0.983      0.994      0.938
               bicycle        122        129          1      0.985      0.995      0.942
             motorbike        487        887      0.989      0.984      0.994      0.941
                   car        330        424      0.986      0.991      0.995      0.975
                   bus          6          6      0.915          1      0.995      0.995
                person         97        220          1      0.955      0.994      0.837
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/Data Warehouse/Vehicle Counting/data-split_yolo11n_100/detect/train


In [ ]:
# # Load a model
# model = YOLO("runs/detect/train/weights/last.pt")  # load a partially trained model

# # Resume training
# results = model.train(resume=True)

In [17]:
print("OK")

OK
